In [10]:
import pandas as pd
import numpy as np
import warnings

# Désactiver les avertissements FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Charger le fichier CSV
df = pd.read_csv("voitures_final_concatene.csv")

# Supprimer les colonnes inutiles
df = df.drop(['Marque', 'Modèle', 'Type', 'Secteur'], axis=1)

# 1. Remplir les NaN dans la colonne 'Origine' et effectuer les remplacements
df['Origine'] = df['Origine'].fillna("Non spécifié")
df['Origine'] = df['Origine'].replace({
    'WW au Maroc': 1,
    'Dédouanée': 2,
    'Non spécifié': 0,
    'Importée neuve': 3,
    'Pas encore dédouanée': 4
})

# 2. Convertir la colonne 'Prix' en float et ajuster l'échelle
df['Prix'] = df['Prix'].astype(float) / 1000

# 3. Remplir les NaN dans la colonne 'Première main' avec la valeur la plus fréquente
mode_premiere_main = df['Première main'].mode()[0]
df['Première main'] = df['Première main'].fillna(mode_premiere_main)

# Transformer en booléen : True si "Oui", False sinon
df['Première main'] = (df['Première main'] == "Oui")

# 4. Remplacer les valeurs spécifiques de 'État' par des entiers
df['État'] = df['État'].replace({
    'Bon': 1, 
    'Très bon': 2, 
    'Excellent': 3, 
    'Neuf': 4,
    'Correct': 0,
    'Pour Pièces': -1,
    'Endommagé': -2
})

# Remplir les NaN dans 'État' avec 0
df['État'] = df['État'].fillna(0)

# 5. Remplir les NaN dans la colonne 'BoiteàV' avec la valeur la plus fréquente (mode)
mode_boiteav = df['BoiteàV'].mode()[0]
df['BoiteàV'] = df['BoiteàV'].fillna(mode_boiteav)

# Créer la nouvelle colonne 'BoiteàV_Auto' (True si Automatique, False sinon)
df['BoiteàV_Auto'] = (df["BoiteàV"] == "Automatique")

# Supprimer l'ancienne colonne 'BoiteàV'
df = df.drop(["BoiteàV"], axis=1)

# 6. Ajouter les colonnes dummies pour la colonne 'Carburant'
df = pd.concat([df, pd.get_dummies(df['Carburant'], prefix='Carburant')], axis=1)

# Supprimer l'ancienne colonne 'Carburant'
df = df.drop(['Carburant'], axis=1)

# Remplir les NaN dans les nouvelles colonnes Carburant_* avec le mode de chaque colonne
carburant_cols = [col for col in df.columns if col.startswith('Carburant')]  # Sélectionner les nouvelles colonnes
for col in carburant_cols:
    mode_val = df[col].mode()[0]  # Trouver le mode de chaque colonne
    df[col] = df[col].fillna(mode_val)  # Remplacer NaN par le mode

# 7. Fonction pour nettoyer la colonne 'Kilométrage'
def clean_kilometrage(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip()

    try:
        if ' - ' in val:
            # Exemple : "10 000 - 15 000"
            bounds = val.split(' - ')
            low = int(bounds[0].replace(" ", ""))
            high = int(bounds[1].replace(" ", ""))
            return (low + high) / 2
        elif 'Moins de' in val:
            # Exemple : "Moins de 5 000"
            number = int(val.replace("Moins de", "").replace(" ", ""))
            return number / 2  # Exemple : 2500
        elif 'Plus de' in val:
            # Exemple : "Plus de 500 000"
            number = int(val.replace("Plus de", "").replace(" ", ""))
            return number + 10000  # Ajoute une marge au-dessus
        else:
            return np.nan
    except:
        return np.nan

# Nettoyer la colonne Kilométrage
df['Kilométrage'] = df['Kilométrage'].apply(clean_kilometrage)

# Remplacer les valeurs manquantes de 'Kilométrage' par la médiane
median_km = df['Kilométrage'].median()
df['Kilométrage'] = df['Kilométrage'].fillna(median_km)

# 8. Transformer la colonne 'équipements' en listes et créer des colonnes pour chaque équipement
equipements_list = df['équipements'].dropna().apply(lambda x: [e.strip() for e in x.split(',')])

equipements_uniques = {'Airbags', 'Verrouillage centralisé à distance', 'Radar de recul', 'ESP', 
                       'Climatisation', 'Système de navigation/GPS', 'Ordinateur de bord', 
                       'Toit ouvrant', 'ABS', 'Vitres électriques', 
                       'Limiteur de vitesse', 'CD/MP3/Bluetooth', 'Sièges cuir', 
                       'Jantes aluminium', 'Régulateur de vitesse', 'Caméra de recul'}

# Créer une colonne pour chaque équipement
for equipement in equipements_uniques:
    df[equipement] = equipements_list.apply(lambda x: equipement in x if isinstance(x, list) else False)

# Supprimer la colonne d'origine 'équipements'
df = df.drop(['équipements'], axis=1)

# 9. Convertir la colonne 'Année' en type numérique (en cas de valeurs non numériques, elles seront converties en NaN)
df['Année'] = pd.to_numeric(df['Année'], errors='coerce')

# 10. Calculer l'âge en soustrayant l'année de fabrication de 2025
df['Age'] = 2025 - df['Année']

# Supprimer la colonne 'Année' maintenant que l'âge est calculé
df = df.drop(['Année'], axis=1)

# 11. Imputation des valeurs manquantes pour chaque colonne selon son type

# 1. Remplacer les NaN dans les colonnes numériques par la médiane
numerical_columns = ['Prix', 'Kilométrage', 'Puissance fiscale', 'Age', 'Nombre de portes']
for col in numerical_columns:
    df[col] = df[col].fillna(df[col].median())

# 2. Remplacer les NaN dans les colonnes booléennes (équipements) par False
boolean_columns = ['Airbags', 'Climatisation', 'Radar de recul', 'ESP', 'Système de navigation/GPS',
                   'Ordinateur de bord', 'Vitres électriques', 'Caméra de recul', 'CD/MP3/Bluetooth',
                   'Verrouillage centralisé à distance', 'ABS', 'Limiteur de vitesse', 'Régulateur de vitesse',
                   'Jantes aluminium', 'Toit ouvrant', 'Sièges cuir']
for col in boolean_columns:
    df[col] = df[col].fillna(False)

# 3. Remplacer les NaN dans les colonnes catégorielles (ex: 'Origine') par le mode (la valeur la plus fréquente)
categorical_columns = ['Origine']
for col in categorical_columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# 12. Vérifier si toutes les valeurs manquantes ont été traitées
print("Valeurs manquantes après imputation :")
print(df.isnull().sum())

# 13. Convertir toutes les colonnes en float64 (les booléens deviendront 0 ou 1, et les entiers seront convertis en float)
df = df.astype('float64')

# 14. Vérifier les types de colonnes après conversion
print("\nTypes de données après conversion :")
print(df.dtypes)


# Enregistrer le DataFrame nettoyé dans un fichier CSV avec un encodage UTF-8-SIG pour assurer la compatibilité avec Excel
df.to_csv("C:/Users/HP G3/Bureau/data_avito_cleann.csv", index=False, encoding='utf-8-sig')

# Afficher les premières lignes du DataFrame nettoyé
df.head()



Valeurs manquantes après imputation :
Prix                                  0
Kilométrage                           0
Nombre de portes                      0
Origine                               0
Première main                         0
Puissance fiscale                     0
État                                  0
BoiteàV_Auto                          0
Carburant_Diesel                      0
Carburant_Electrique                  0
Carburant_Essence                     0
Carburant_Hybride                     0
Carburant_LPG                         0
Sièges cuir                           0
Toit ouvrant                          0
ESP                                   0
Système de navigation/GPS             0
Vitres électriques                    0
Ordinateur de bord                    0
Caméra de recul                       0
CD/MP3/Bluetooth                      0
Verrouillage centralisé à distance    0
Radar de recul                        0
ABS                                   0
Li

,Prix,Kilométrage,Nombre de portes,Origine,Première main,Puissance fiscale,État,BoiteàV_Auto,Carburant_Diesel,Carburant_Electrique,...,CD/MP3/Bluetooth,Verrouillage centralisé à distance,Radar de recul,ABS,Limiteur de vitesse,Airbags,Régulateur de vitesse,Jantes aluminium,Climatisation,Age
0,140.0,124999.5,5.0,1.0,1.0,6.0,3.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,6.0
1,570.0,134999.5,5.0,1.0,0.0,12.0,3.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
2,268.0,7499.5,5.0,1.0,1.0,6.0,3.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,124.0,224999.5,5.0,1.0,0.0,12.0,3.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,13.0
4,400.0,144999.5,5.0,3.0,1.0,12.0,2.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0
